In [4]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, AutoModelForCausalLM
import json, os, shutil, re, random, io, requests, ctypes, sys, time, struct
from datasets import load_dataset,load_from_disk
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
import pandas as pd
file_path = "../GRPO_data/val_prompt2.csv"
# model_path = "/home/LLM/models/Qwen2.5-0.5B-Instruct"
df = pd.read_csv(file_path)

In [6]:
model_path = "../SFT_only_model-1.5B"
num_pre_Q = 1

In [7]:
sampling_params = SamplingParams(n=num_pre_Q, temperature=0.4, max_tokens=2048)
vllm_gen = LLM(model=model_path, gpu_memory_utilization=0.9)
tokenizer = AutoTokenizer.from_pretrained(model_path)

INFO 06-02 17:00:06 [config.py:2832] Downcasting torch.float32 to torch.float16.
INFO 06-02 17:00:12 [config.py:689] This model supports multiple tasks: {'classify', 'generate', 'score', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 06-02 17:00:12 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-02 17:00:14 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='../SFT_only_model-1.5B', speculative_config=None, tokenizer='../SFT_only_model-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observabi

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.87it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.75s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.57s/it]



INFO 06-02 17:00:18 [loader.py:458] Loading weights took 3.23 seconds
INFO 06-02 17:00:19 [gpu_model_runner.py:1291] Model loading took 2.8876 GiB and 3.382207 seconds
INFO 06-02 17:00:25 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/28e21bda15/rank_0_0 for vLLM's torch.compile
INFO 06-02 17:00:25 [backends.py:426] Dynamo bytecode transform time: 6.05 s
INFO 06-02 17:00:27 [backends.py:132] Cache the graph of shape None for later use
INFO 06-02 17:00:46 [backends.py:144] Compiling a graph for general shape takes 20.77 s
INFO 06-02 17:00:56 [monitor.py:33] torch.compile takes 26.82 s in total
INFO 06-02 17:00:57 [kv_cache_utils.py:634] GPU KV cache size: 1,142,048 tokens
INFO 06-02 17:00:57 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 34.85x
INFO 06-02 17:01:12 [gpu_model_runner.py:1626] Graph capturing finished in 15 secs, took 0.45 GiB
INFO 06-02 17:01:12 [core.py:163] init engine (profile, create kv cache, warmup model) t

In [8]:
system_prompt = """
You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> 
"""
print(system_prompt)


You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
<answer>
###Tables: (Related Table);
###Columns: (Related Columns from Tables);
</answer> 



In [9]:
def make_user_prompt(question, database_schema):
    return f"<question>{question}</question>\n<database>{database_schema}</database>"

In [10]:
def gen_answers(prompts):
    tip_text = []
    for x in prompts:
        tip_text.append(tokenizer.apply_chat_template([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": x}], tokenize=False, add_generation_prompt=True))
    voutputs = vllm_gen.generate(tip_text, sampling_params, use_tqdm=False)
    answers = [];  ans_token_ids = []
    for v in voutputs:
        for z in v.outputs: 
            answers.append(z.text)
            ans_token_ids.append(z.token_ids)
    return answers, ans_token_ids

In [11]:
# from tqdm import tqdm
sample_list = []
correct_list = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    correct_list.append(row["target_schema"])
    user_prompt = [make_user_prompt(row["question"],row["database_schema"])]
#     print(user_prompt)
#     print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    response = gen_answers(user_prompt)[0][0]
    sample_list.append(response)
    if index % 100 ==0 :
        print(response)
        print(row["target_schema"])
        print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

  0%|                                                                                                          | 1/1004 [00:00<08:20,  2.01it/s]

<answer>
###Tables: vocalist;
###Columns: vocalist.name, vocalist.age, vocalist.country, vocalist.technician, vocalist.song_name, vocalist.consultant, vocalist.damaged, vocalist.repair, vocalist.artist, vocalist.album, vocalist.track, vocalist.unit_price;
</answer>
###Tables: singer;
###Columns: singer.name, singer.song_release_year, singer.country, singer.singer_id, singer.song_name, singer.age;

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


 10%|██████████▌                                                                                             | 102/1004 [00:30<05:30,  2.73it/s]

<answer>
###Tables: car_makers, car_names, cars_data;
###Columns: car_makers.id, car_makers.maker, car_names.year, car_names.maker, cars_data.year, cars_data.maker;
</answer>
###Tables: car_makers, model_list, car_names, cars_data;
###Columns: car_makers.maker, car_makers.id, model_list.maker, model_list.model, car_names.model, cars_data.year, cars_data.id;

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


 20%|████████████████████▉                                                                                   | 202/1004 [00:50<01:56,  6.91it/s]

...

 90%|█████████████████████████████████████████████████████████████████████████████████████████████▍          | 902/1004 [03:25<00:23,  4.38it/s]

<answer>
###Tables: owners, dogs;
###Columns: owners.first_name, owners.last_name, owners.owner_id, dogs.owner_id;
</answer>
###Tables: dogs, owners;
###Columns: dogs.owner_id, owners.last_name, owners.first_name, owners.owner_id;

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████▋| 1001/1004 [03:44<00:00,  3.19it/s]

<answer>
###Tables: other_available_features;
###Columns: other_available_features.feature_description, other_available_features.feature_name, other_available_features.feature_id, other_available_features.other_property_id, other_available_features.apt_feature_1, other_available_features.apt_feature_2, other_available_features.hse_feature_1, other_available_features.other_feature_1, other_available_features.apt_feature_3, other_available_features.hse_feature_2, other_available_features.other_property_id, other_available_features.feature_type_code;
</answer>
###Tables: other_available_features;
###Columns: other_available_features.feature_type_code, other_available_features.feature_description, other_available_features.feature_id, other_available_features.feature_name;

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1004/1004 [03:45<00:00,  4.46it/s]


In [12]:
print(len(sample_list))
print(len(correct_list))
print(correct_list[0])

1004
1004
###Tables: singer;
###Columns: singer.name, singer.song_release_year, singer.country, singer.singer_id, singer.song_name, singer.age;



In [13]:
import pandas as pd
write_dict = {}
write_dict["sample"] = sample_list
write_dict["correct"] = correct_list

csv_filename = 'SFT_only.csv'
# 将字典转换为 DataFrame
df = pd.DataFrame(write_dict)
 
# 写入 CSV 文件
df.to_csv(csv_filename, index=False)
 
print(f"字典已成功写入到 {csv_filename}")

字典已成功写入到 SFT_only.csv


In [14]:
def check_socore_label(ground_truth ,answer):

    # get ground truth      
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)
    
    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    
    truth_tables = {s.lower() for s in truth_tables}
    truth_columns = {s.lower() for s in truth_columns}
    
    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    
    pre_tables = {s.lower() for s in pre_tables}
    pre_columns = {s.lower() for s in pre_columns}

    table_format_len = len(truth_tables - (truth_tables - pre_tables))
    column_format_len = len(truth_columns - (truth_columns - pre_columns))
    
    
    table_format_acc =  table_format_len / len(truth_tables)
    column_format_acc =  column_format_len / len(truth_columns)
    
   
    return table_format_acc, column_format_acc

In [15]:
def strict_format_label(ground_truth ,answer):

    # get ground truth      
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)
    
    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    
    
    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    
    table_format = 0
    column_format = 0
    if pre_tables ==  truth_tables:
        table_format = 1
    
    if pre_columns ==  truth_columns:
        column_format = 1
   
    return table_format, column_format

In [16]:
table_acc_list = []
columns_acc_list = []
for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    table_format_reward, column_format_reward = check_socore_label(correct_list[index],sample_list[index])
    print("table_format_reward:",table_format_reward)
    print("column_format_reward:",column_format_reward)
    table_acc_list.append(table_format_reward)
    columns_acc_list.append(column_format_reward)
    print("+++++++++++++++++++++++++++")
#     gen_len_reward(sample_list[index])

table_format_reward: 0.0
column_format_reward: 0.0
+++++++++++++++++++++++++++
table_format_reward: 0.0
column_format_reward: 0.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 0.0
column_format_reward: 0.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 0.0
column_format_reward: 0.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 0.3333333333333333
+++++++++++++++++++++++++++
table_format_reward: 0.0
column_format_reward: 0.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 0.0
column_format_reward: 0.0
+++++++++++++++++++++++++++
table_format_reward: 0.0
column_format_reward: 0.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_forma

In [17]:
print("avg_table_format:",np.sum(table_acc_list)/len(table_acc_list))
print("avg_column_format:",np.sum(columns_acc_list)/len(columns_acc_list))

avg_table_format: 0.8303452855245683
avg_column_format: 0.5941190594763801


In [18]:
table_format_list = []
column_format_list = []
for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    table_format, column_format = strict_format_label(correct_list[index],sample_list[index])
    print("table_format:",table_format)
    print("column_format:",column_format)
    table_format_list.append(table_format)
    column_format_list.append(column_format)
    print("+++++++++++++++++++++++++++")
#     gen_len_reward(sample_list[index])

table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_f

In [19]:
print("avg_table_format:",np.sum(table_format_list)/len(table_format_list))
print("avg_column_format:",np.sum(column_format_list)/len(column_format_list))

avg_table_format: 0.648406374501992
avg_column_format: 0.31175298804780877


In [20]:
# dts-bench

In [21]:
def get_Table_columns(ground_truth ,answer):
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)

    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    truth_tables = {s.lower() for s in truth_tables}
    truth_columns = {s.lower() for s in truth_columns}

    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    pre_tables = {s.lower() for s in pre_tables}
    pre_columns = {s.lower() for s in pre_columns}
    return truth_tables, truth_columns, pre_tables, pre_columns

In [22]:
total_samples = len(sample_list)

table_total_accuracy = 0
table_filtered_accuracy = 0
table_total_precision = 0
table_total_recall = 0


columns_total_accuracy = 0
columns_filtered_accuracy = 0
columns_total_precision = 0
columns_total_recall = 0


for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    precision = 0
    recall = 0
    truth_tables, truth_columns, pre_tables, pre_columns = get_Table_columns(correct_list[index],sample_list[index])
    if set(pre_tables) == set(truth_tables):
        table_total_accuracy += 1
    table_true_positives = len(set(pre_tables) & set(truth_tables))
    table_false_positives = len(set(pre_tables) - set(truth_tables))
    table_false_negatives = len(set(truth_tables) - set(pre_tables))
    
    if table_true_positives == len(truth_tables):
        table_filtered_accuracy += 1
    
    if len(pre_tables) > 0:
        precision = table_true_positives / (table_true_positives + table_false_positives)
        recall = table_true_positives / (table_true_positives + table_false_negatives)
    
    table_total_precision += precision
    table_total_recall += recall
    
    ########################cloumns####################################
    precision = 0
    recall = 0
    if set(pre_columns) == set(truth_columns):
        columns_total_accuracy += 1
    columns_true_positives = len(set(pre_columns) & set(truth_columns))
    columns_false_positives = len(set(pre_columns) - set(truth_columns))
    columns_false_negatives = len(set(truth_columns) - set(pre_columns))
    
    if columns_true_positives == len(truth_columns):
        columns_filtered_accuracy += 1
    
    if len(pre_columns) > 0:
        precision = columns_true_positives / (columns_true_positives + columns_false_positives)
        recall = columns_true_positives / (columns_true_positives + columns_false_negatives)
    
    columns_total_precision += precision
    columns_total_recall += recall

In [23]:
avg_precision = table_total_accuracy / total_samples
avg_recall = table_total_recall / total_samples

# Calculate total accuracy
accuracy = table_total_accuracy / total_samples
filtered_accuracy = table_filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.648406374501992
Filtered Accuracy: 0.75
Average Precision: 0.648406374501992
Average Recall: 0.8303452855245682


In [24]:
# Calculate average precision and recall
avg_precision = columns_total_accuracy / total_samples
avg_recall = columns_total_recall / total_samples

# Calculate total accuracy
accuracy = columns_total_accuracy / total_samples
filtered_accuracy = columns_filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.31175298804780877
Filtered Accuracy: 0.4243027888446215
Average Precision: 0.31175298804780877
Average Recall: 0.5941190594763803
